In [98]:
# !pip install langdetect

In [99]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import requests
from time import sleep
import random

from langdetect import detect, DetectorFactory

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [100]:
test_df = pd.read_csv('~/Downloads/test.csv')
test_df['score'] = [random.randint(3, 5) for _ in range(len(test_df))]

test_df = test_df[['title', 'user_review', 'score']]

In [101]:
# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove escape characters
    text = re.sub(r'\\[a-z]', '', text)
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Join tokens back into a string
    cleaned_text = ' '.join(tokens)
    return cleaned_text

# Seed for consistent language detection
DetectorFactory.seed = 0

# Function to detect language
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [102]:
#Text cleaning

#only English reviews
test_df['is_english'] = test_df['user_review'].apply(is_english)
english_reviews_df = test_df[test_df['is_english'] == True]

# Drop the 'is_english' helper column if needed
english_reviews_df = english_reviews_df.drop(columns=['is_english'])

# Apply the cleaning function to the dataset
english_reviews_df['cleaned_review'] = english_reviews_df['user_review'].apply(clean_text)

#Dropping original review column
english_reviews_df = english_reviews_df.drop(columns=['user_review'])

# Change h1z1 to Z1 Battle Royale
english_reviews_df.loc[english_reviews_df['title'] == 'H1Z1', 'title'] = 'Z1 Battle Royale'

In [103]:
english_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7957 entries, 0 to 8044
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   title           7957 non-null   object
 1   score           7957 non-null   int64 
 2   cleaned_review  7957 non-null   object
dtypes: int64(1), object(2)
memory usage: 248.7+ KB


In [104]:
english_reviews_df.to_csv('ds4300_final.csv', index=False)